<a href="https://colab.research.google.com/github/CristianViSa/Biometria/blob/master/lstm_mejorada_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datatable

     |████████████████████████████████| 96.9 MB 105 kB/s 


Imports


In [ ]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime
import time
import keras.backend as K
import math
import pickle
from pathlib import Path

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Bidirectional, Dropout, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, roc_curve, auc, mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
from numpy import concatenate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import regularizers
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import datatable as dt 

In [ ]:
mape = tf.keras.losses.MeanAbsolutePercentageError()
def calc_mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100


def calc_smape(A, F):
  A, F = np.array(A), np.array(F)
  return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


Paths y lectura de datos

In [ ]:
full_data_path = "/content/drive/MyDrive/datos_web/otros/test3.csv"
ordered_data_path = "/content/drive/MyDrive/datos_web/otros/cleaned_ordered.csv"
# ordered_data_path = "/content/drive/MyDrive/datos_web/otros/cleaned_ordered_2.csv"
final_data_path = "/content/drive/MyDrive/datos_web/otros/final_data.csv"

# test_cleaned_full_path = "/content/drive/MyDrive/datos_web/otros/full_cleaned.csv"

In [ ]:
final_data = dt.fread(final_data_path).to_pandas()
dates = pd.to_datetime(final_data["C0"].values)
final_data.drop("C0", axis = 1, inplace=True)
final_data.set_index(dates, inplace=True)

# final_data.set_
ordered_data = dt.fread(ordered_data_path).to_pandas()
ordered_data.drop("C0", axis = 1, inplace=True)


Calles y sus entradas. Seleccionar datos del conjunto

In [ ]:
avcid = ["A145", "A70", "A71", "A72", "A69"]

# alameda = ["A174", "A345" ,"A266", "A277", "A166", "A173"]
alameda = ["A174", "A345" ,"A173"]
# genaro = ["B24", "B118", "B81"]
genaro = ["B24", "B81", "B118", "B57"]
# avmed = ["A131", "A179", "A104", "A88", "A351" ]
avmed = ["A131", "A179", "A351" ]
# justopastor = ["A415", "A359", "A92", "A230","A231"]
justopastor = ["A415", "A359", "A92", "A231"]
# baron = ["A98", "B29", "B10"]
baron = ["A98", "B10"]
candela = ["A211", "A50", "A49", "A231", "A253", "A91", "A92"]
aragon = ["A47", "A33", "A34", "A5"]
ausias = ["A27", "A29", "A197","A28"] # probar a cambiar a197 por A187
puentearagon = ["A121", "A8", "A34","A7", "A278"]
baleares = ["A61", "A384", "A241", "A270"]
blasco = ["A50", "A92", "A47", "A49"]

Para crear la serie temporal, cambiar "cols" en funcion de lo que se quiera

In [ ]:
def festives(serie):

  festivos = ["2021-03-19", "2021-04-02", "2021-04-05", "2021-04-12", "2021-05-01", "2021-06-24"]
  dates = serie.index.date
  festivos = []
  for d in dates:
    if d in festivos:
      festivos.append(1)
    else:
      festivos.append(0)

  return festivos


In [ ]:
cols = avcid
datos = final_data[cols]
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos_pre = datos["2021-03-15 00:00:00":"2021-05-09 23:00:00"]
datos_post = datos["2021-05-10 00:00:00":"2021-06-20 23:00:00"]
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos.fillna(0.1, inplace=True)
datos[datos == 0] = 0.1
datos['festivos'] = festives(datos)
# Para entrenar modelo final (NO AJUSTAR PARAMETROS CON ESTE CONJUNTO)
train = datos["2021-03-15 00:00:00":"2021-06-06 23:00:00"]
test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# Para el proceso de ajuste
# train = datos["2021-03-15 00:00:00":"2021-05-16 23:00:00"]
val = datos["2021-05-17 00:00:00":"2021-06-06 23:00:00"]
# test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# cols = ["A72"]
# datos = final_data[cols]
# datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
# # Para entrenar modelo final (NO AJUSTAR PARAMETROS CON ESTE CONJUNTO)
# train = datos["2021-03-15 00:00:00":"2021-06-06 23:00:00"]
# # test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]
# test = datos["2021-06-07 00:00:00":"2021-06-08 23:00:00"]
# val = test

In [ ]:
train

,A145,A70,A71,A72,A69,festivos
2021-03-15 00:00:00,75.0,10.0,199.0,231.0,0.1,0
2021-03-15 01:00:00,42.0,0.1,210.0,231.0,0.1,0
2021-03-15 02:00:00,42.0,0.1,210.0,231.0,0.1,0
2021-03-15 03:00:00,42.0,0.1,210.0,231.0,0.1,0
2021-03-15 04:00:00,42.0,0.1,210.0,231.0,0.1,0
...,...,...,...,...,...,...
2021-06-06 19:00:00,548.0,1027.0,1852.0,2722.0,1035.0,0
2021-06-06 20:00:00,467.0,1320.0,1642.0,3191.0,1402.0,0
2021-06-06 21:00:00,676.0,1005.0,1391.0,2527.0,1350.0,0
2021-06-06 22:00:00,519.0,272.0,909.0,1761.0,691.0,0


Crear la serie temporal.

2 formas diferentes :
  1 - Método create_dataset
  2 - Método series_to_supervised
  

In [ ]:
# # Convertir la serie temporal en datos supervisados
# Serie sin tener en cuenta informacion adicional, solo la hora anterior
def series_to_supervised(data, keys, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('%s(t-%d)' % (keys[j], i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('%s(t)' % (keys[j])) for j in range(n_vars)]
    else:
      names += [('%s(t+%d)' % (keys[j], i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(0, inplace=True)
  return agg


features = train.shape[1]
timesteps = 2
outputs = 1

## Cambiar en funcion de lo que se quiera
# train = weekdays_train
# test = weekdays_test
# val = weekends_val
# train = weekends_train
# # test = weekends_test
train_keys = train.keys()
test_keys = test.keys()
val_keys = val.keys()
print(test.iloc[0:24])
scaler = MinMaxScaler(feature_range=(0.1,1))
scaler = scaler.fit(train)
train_values = scaler.transform(train)
test_values = scaler.transform(test)
val_values = scaler.transform(val)

# train_values = train.values
# test_values = test.values
# val_values = val.values

serie_train = series_to_supervised(train_values, train_keys, n_in=1, n_out=2)
serie_test = series_to_supervised(test_values, test_keys,  n_in=1, n_out=2)
serie_val = series_to_supervised(val_values, val_keys, n_in=1, n_out=2)

x_train, y_train = serie_train.values[:, :-features], serie_train.values[:, -features:]
x_test, y_test = serie_test.values[:, :-features], serie_test.values[:, -features:]
x_val, y_val = serie_val.values[:, :-features], serie_val.values[:, -features:]

##############  Resize de los arrays ##########
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
x_val = np.asarray(x_val).astype('float32')

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
y_val = np.asarray(y_val).astype('float32')

x_train = np.reshape(x_train, (x_train.shape[0], timesteps, features))
x_test = np.reshape(x_test, (x_test.shape[0], timesteps, features))
x_val = np.reshape(x_val, (x_val.shape[0], timesteps, features))


                       A145     A70     A71     A72     A69  festivos
2021-06-07 00:00:00   171.0   104.0   202.0   315.0   136.0         0
2021-06-07 01:00:00   148.0    63.0   162.0   270.0    89.0         0
2021-06-07 02:00:00   116.0    63.0   155.0   194.0    63.0         0
2021-06-07 03:00:00    88.0     0.1   270.0    21.0     0.1         0
2021-06-07 04:00:00    88.0     0.1   411.0    21.0     0.1         0
2021-06-07 05:00:00    88.0     0.1   552.0    21.0     0.1         0
2021-06-07 06:00:00    88.0     0.1   693.0    21.0     0.1         0
2021-06-07 07:00:00   392.0   480.0  1275.0  2150.0   795.0         0
2021-06-07 08:00:00  1585.0  2137.0  3972.0  7068.0  3277.0         0
2021-06-07 09:00:00  1396.0  2175.0  3513.0  6633.0  3097.0         0
2021-06-07 10:00:00   759.0  1120.0  2686.0  4785.0  1852.0         0
2021-06-07 11:00:00   894.0  1395.0  2478.0  4644.0  1944.0         0
2021-06-07 12:00:00   872.0  1332.0  2531.0  4795.0  2232.0         0
2021-06-07 13:00:00 

In [ ]:
y_train

array([[0.1209163 , 0.1       , 0.14706396, 0.12857932, 0.1       ,
        0.1       ],
       [0.1209163 , 0.1       , 0.14706396, 0.12857932, 0.1       ,
        0.1       ],
       [0.1209163 , 0.1       , 0.14706396, 0.12857932, 0.1       ,
        0.1       ],
       ...,
       [0.4374064 , 0.42072415, 0.41186875, 0.41387546, 0.4583911 ,
        0.1       ],
       [0.35903266, 0.18677968, 0.30379432, 0.3186939 , 0.2834302 ,
        0.1       ],
       [0.17033668, 0.25188836, 0.21746929, 0.20996824, 0.27758932,
        0.1       ]], dtype=float32)

In [ ]:
y_train = np.delete(y_train, -1, axis=1)
y_test = np.delete(y_test, -1, axis=1)
y_val = np.delete(y_val, -1, axis=1)

In [ ]:
y_train

array([[0.1209163 , 0.1       , 0.14706396, 0.12857932, 0.1       ],
       [0.1209163 , 0.1       , 0.14706396, 0.12857932, 0.1       ],
       [0.1209163 , 0.1       , 0.14706396, 0.12857932, 0.1       ],
       ...,
       [0.4374064 , 0.42072415, 0.41186875, 0.41387546, 0.4583911 ],
       [0.35903266, 0.18677968, 0.30379432, 0.3186939 , 0.2834302 ],
       [0.17033668, 0.25188836, 0.21746929, 0.20996824, 0.27758932]],
      dtype=float32)

Creación del modelo

In [ ]:
####### Crear el modelo  ##########
def create_model(batch, lr, epochs, show_graph):
  model = Sequential()
  model.add(LSTM(64, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True, kernel_regularizer=regularizers.l2(1e-6),
      bias_regularizer=regularizers.l2(1e-6),
      activity_regularizer=regularizers.l2(0.01)
  ))
  model.add(LSTM(32, activation='relu', return_sequences=False, kernel_regularizer=regularizers.l2(1e-6),
      bias_regularizer=regularizers.l2(1e-6),
      activity_regularizer=regularizers.l2(0.01)))
  model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(1e-6),
      bias_regularizer=regularizers.l2(1e-6),
      activity_regularizer=regularizers.l2(0.01)))
  model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-6),
      bias_regularizer=regularizers.l2(1e-6),
      activity_regularizer=regularizers.l2(0.01)))
  model.add(Dense(features-1))
  model.summary()
  
  if show_graph:
    tf.keras.utils.plot_model(model, show_shapes=True)
  return model

Entrenar los modelos. Guarda la precisión obtenida con el conjunto de validación para su posterior análisis

In [ ]:
# Entrenar los modelos
# lrs = [0.1, 0.01, 0.001, 0.0001, 0.00001]
# batches = [1, 2, 4, 8, 16, 24, 32, 64, 128, 256]
# Best configuration
lrs = [0.0001]
batches = [24]
epochs = 400
results = {}
metrics = ["mse", "mae", "mape"]

for lr in lrs:
  for batch in batches:
    for loss in metrics:
      combination = str(lr) + "-" + str(batch) + "-" + str(loss)
      model = create_model(batch, lr, epochs, True)
      model.compile(metrics=metrics, loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=lr))

      # Path for the weights
      name = "/content/drive/MyDrive/datos_web/otros/" + combination
      path = Path(name)
      if not path.exists():
        path.mkdir(parents=True)
      filepath = name + "/weights.best.hdf5"
      es = EarlyStopping(monitor=loss, mode='min', verbose=1, patience=2)
      checkpoint = ModelCheckpoint(filepath, monitor=loss, verbose=1, save_best_only=True, mode='min')
      callbacks_list = [checkpoint, es]
      history = model.fit(
        x_train, y_train, 
        epochs=epochs, 
        batch_size=batch, 
        validation_data=(x_val, y_val),
        callbacks=callbacks_list,
        shuffle=False,
        verbose=1)
      plt.title("MAE")
      plt.xlabel("Épocas")
      plt.ylabel("MAE")
      plt.plot(history.history['mae'], label='entrenamiento')
      plt.plot(history.history['val_mae'], label='test')
      plt.legend()
      # plt.show()
      out_file = name + "/MAE.png"
      plt.savefig(out_file)
      plt.close()

      plt.title("MSE")
      plt.xlabel("Épocas")
      plt.ylabel("MSE")
      plt.plot(history.history['mse'], label='entrenamiento')
      plt.plot(history.history['val_mse'], label='test')
      plt.legend()
      # plt.show()
      out_file = name + "/MSE.png"
      plt.savefig(out_file)
      plt.close()
      
      plt.title("MAPE")
      plt.xlabel("Épocas")
      plt.ylabel("MAPE")
      plt.plot(history.history['mape'], label='entrenamiento')
      plt.plot(history.history['val_mape'], label='validacion')
      plt.legend()
      # plt.show()
      out_file = name + "/MAPE.png"
      plt.savefig(out_file)
      plt.close()
      # Write to a file the best loss obtained
      with open("/content/drive/MyDrive/datos_web/otros/results.txt", "a") as file_pi:
        l = np.array(history.history['loss'])
        l.sort()
        out = "Results for " + combination + "\n"
        file_pi.write(out)
        file_pi.write(str(l[0]))
        file_pi.write("\n ------------------------------------------------- \n")
        file_pi.close()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 64)             18176     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total params: 31,829
Trainable params: 31,829
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
84/84 [==============================] - 4s 12ms/step - loss: 0.1639 - mse: 0.1630 - mae: 0.3321 - mape: 9

In [ ]:
# Hacer predicciones y mostrarlas

# Cambiar el path correspondiente
path = "/content/drive/MyDrive/datos_web/otros/0.0001-24-mae/weights.best.hdf5"
model.load_weights(path)

# Hacer predicciones
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)
val_predict = model.predict(x_val)

train_festives = np.array(train['festivos'][2:])
test_festives = np.array(test['festivos'][2:])
val_festives = np.array(val['festivos'][2:])

train_predict = np.column_stack((train_predict, train_festives))
test_predict = np.column_stack((test_predict, test_festives))
val_predict = np.column_stack((val_predict, val_festives))


y_train = np.column_stack((y_train, train_festives))
y_test = np.column_stack((y_test, test_festives))
y_val = np.column_stack((y_val, val_festives))

train_predict_inversed = scaler.inverse_transform(train_predict)
test_predict_inversed = scaler.inverse_transform(test_predict)  
val_predict_inversed = scaler.inverse_transform(val_predict)  

y_test_inversed = scaler.inverse_transform(y_test)  
y_train_inversed = scaler.inverse_transform(y_train)
y_val_inversed = scaler.inverse_transform(y_val)

total_train_rmse = 0
total_train_mae = 0
total_train_mse = 0

total_test_rmse = 0
total_test_mae = 0
total_test_mse = 0
total_test_mape = 0

for i in range(test_predict.shape[1]):
  train_score = math.sqrt(mean_squared_error(y_train[:, i], train_predict[:,  i]))
  train_mse_score = mean_squared_error(y_train[:, i], train_predict[:,  i])
  train_mae_score = mean_absolute_error(y_train[:, i], train_predict[:,  i])

  print('Train Score: %.5f RMSE' % (train_score))
  print('Train Score: %.5f MAE' % (train_mae_score))
  print('Train Score: %.5f MSE' % (train_mse_score))

  total_train_rmse += train_score
  total_train_mae += train_mae_score
  total_train_mse += train_mse_score

  test_score = math.sqrt(mean_squared_error(y_test[:, i], test_predict[:,  i]))
  test_mae_score = mean_absolute_error(y_test[:, i], test_predict[:,  i])
  test_mse_score = mean_squared_error(y_test[:, i], test_predict[:,  i])
  test_mape_score = calc_mape(y_test[:, i], test_predict[:,  i])
  test_smape_score = calc_smape(y_test[:, i], test_predict[:,  i])
  print('Test Score: %.5f RMSE' % (test_score))
  print('Test Score: %.5f MAE' % (test_mae_score))
  print('Test Score: %.5f MSE' % (test_mse_score))
  print('Test Score: %.5f MAPE' % (test_mape_score))
  print('Test Score: %.5f sMAPE' % (test_smape_score))
  total_test_rmse += test_score
  total_test_mae += test_mae_score
  total_test_mse += test_mse_score
  total_test_mape += test_mape_score
  total_test_mape += test_smape_score
  fig, ax = plt.subplots()
  predicted_values = test_predict_inversed[:, i]
  real_values = y_test_inversed[:, i]
  horas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
           20, 21, 22, 23]
  # ax.plot(horas, real_values[24:48], "-x", color = 'red', label = 'Real')
  # ax.plot(horas, predicted_values[24:48], "-x", color = 'blue', label = 'Forecasted')
  ax.plot(real_values, color = 'red', label = 'Reales')
  ax.plot(predicted_values, color = 'blue', label = 'Predichos')
  # ax.set_title(train_keys[i])
  if len(ordered_data[ordered_data.ATA==train_keys[i]]) > 0:
    name = ordered_data[ordered_data.ATA==train_keys[i]].desc.iloc[0]
  else:
    name = train_keys[i]
  ax.set_xlabel('Hour')
  ax.set_title(name)
  ax.tick_params(labelsize ='medium', pad = 5, direction = 'out', rotation = 45)
  ax.set_ylabel('Vehicles')
  ax.set_xticks(np.arange(len(horas)))
  ax.legend()
  plt.xticks(rotation='vertical')
  plt.show()
  print("ATA -------> ", train_keys[i])
  print("-------------------- ACABA AQUI, CALLE NUEVA -----------------------")
print('Total train Score: %.4f RMSE' % (total_train_rmse))
print('Total train Score: %.4f MAE' % (total_train_mae))
print('Total train Score: %.4f MSE' % (total_train_mse))

print('Total test Score: %.4f RMSE' % (total_test_rmse))
print('Total test Score: %.4f MAE' % (total_test_mae))
print('Total test Score: %.4f MSE' % (total_test_mse))
print('Total test Score: %.4f MSE' % (total_test_mape))


print('AVG train Score: %.4f RMSE' % (total_train_rmse / features))
print('AVG train Score: %.4f MAE' % (total_train_mae / features))
print('AVG train Score: %.4f MSE' % (total_train_mse / features))


print('AVG test Score: %.4f RMSE' % (total_test_rmse / features))
print('AVG test Score: %.4f MAE' % (total_test_mae / features))
print('AVG test Score: %.4f MSE' % (total_test_mse / features))
print('AVG test Score: %.4f MAPE' % (total_test_mape / features))

score = calc_mape(y_test_inversed, test_predict_inversed)
print("TOTAL MAPE %f " % (score))

ValueError: ignored